## Вариант 3. Контроль количества параметров

**Цель:** Создание компактной сети.

- Создайте архитектуру, используя **не более 50 000 параметров**.
- **Условие:** Один слой должен быть **1x1 сверткой**, чтобы уменьшить число каналов.
- **Эксперимент:** Подсчитайте количество параметров каждого слоя и убедитесь, что общая сумма не превышает лимита.

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CompactNet(nn.Module):
    def __init__(self, input_channels=3, output_channels=10):
        super(CompactNet, self).__init__()
        
        # Параметры для контроля
        self.max_params = 50000
        self.total_params = 0
        self.layer_params = {}
        
        # Слой 1: Conv 3x3 с небольшим количеством каналов
        self.conv1 = nn.Conv2d(input_channels, 16, kernel_size=3, padding=1)
        self._count_params('conv1')
        
        # Слой 2: 1x1 свертка для уменьшения каналов
        self.conv1x1 = nn.Conv2d(16, 8, kernel_size=1)
        self._count_params('conv1x1')
        
        # Слой 3: Conv 3x3
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self._count_params('conv2')
        
        # Слой 4: Depthwise separable свертка
        self.depthwise = nn.Conv2d(16, 16, kernel_size=3, padding=1, groups=16)
        self.pointwise = nn.Conv2d(16, 32, kernel_size=1)
        self._count_params('depthwise')
        self._count_params('pointwise')
        
        # Слой 5: Финальная свертка
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self._count_params('conv3')
        
        # Global Average Pooling
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        
        # Финальный классификатор
        self.fc = nn.Linear(64, output_channels)
        self._count_params('fc')
        
        # Проверка лимита параметров
        self._verify_parameter_limit()
        
        # Слои нормализации
        self.bn1 = nn.BatchNorm2d(16)
        self.bn2 = nn.BatchNorm2d(8)
        self.bn3 = nn.BatchNorm2d(16)
        self.bn4 = nn.BatchNorm2d(32)
        self.bn5 = nn.BatchNorm2d(64)
        
    def _count_params(self, layer_name):
        """Подсчет параметров слоя"""
        layer = getattr(self, layer_name)
        if hasattr(layer, 'weight'):
            num_params = layer.weight.numel()
            if hasattr(layer, 'bias') and layer.bias is not None:
                num_params += layer.bias.numel()
            self.total_params += num_params
            self.layer_params[layer_name] = num_params
    
    def _verify_parameter_limit(self):
        """Проверка лимита параметров"""
        print("=" * 50)
        print("АНАЛИЗ КОЛИЧЕСТВА ПАРАМЕТРОВ")
        print("=" * 50)
        
        # Вывод параметров по слоям
        for layer_name, params in self.layer_params.items():
            print(f"{layer_name:15}: {params:7,} параметров")
        
        print("-" * 50)
        print(f"Общее количество параметров: {self.total_params:,}")
        print(f"Лимит параметров: {self.max_params:,}")
        print(f"Использовано: {self.total_params/self.max_params*100:.1f}% лимита")
        
        # Проверка лимита
        if self.total_params > self.max_params:
            raise ValueError(f"Превышен лимит параметров! {self.total_params} > {self.max_params}")
        else:
            print("✓ Лимит параметров соблюден!")
    
    def forward(self, x):
        # Блок 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)  # 32x32 -> 16x16
        
        # Блок 2: 1x1 свертка
        x = self.conv1x1(x)
        x = self.bn2(x)
        x = F.relu(x)
        
        # Блок 3
        x = self.conv2(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)  # 16x16 -> 8x8
        
        # Блок 4: Depthwise separable
        x = self.depthwise(x)
        x = self.pointwise(x)
        x = self.bn4(x)
        x = F.relu(x)
        
        # Блок 5
        x = self.conv3(x)
        x = self.bn5(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)  # 8x8 -> 4x4
        
        # Global Average Pooling
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        
        # Классификация
        x = self.fc(x)
        
        return x







In [7]:
# Эксперимент и тестирование
def run_parameter_experiment():
    print("ЭКСПЕРИМЕНТ: КОНТРОЛЬ КОЛИЧЕСТВА ПАРАМЕТРОВ")
    print("="*60)
    
    # Создаем модели
    print("\n1. Компактная сеть (CompactNet):")
    model = CompactNet(input_channels=3, output_channels=10)

    
    # Тестируем на случайном тензоре
    print("\n\n" + "="*60)
    print("ТЕСТИРОВАНИЕ ПРОИЗВОДИТЕЛЬНОСТИ")
    print("="*60)
    
    input_tensor = torch.randn(4, 3, 32, 32)  # batch=4, channels=3, 32x32
    
    # Тест первой модели
    output1 = model(input_tensor)
    print(f"\nCompactNet:")
    print(f"  Вход: {input_tensor.shape}")
    print(f"  Выход: {output1.shape}")
    print(f"  Параметры: {sum(p.numel() for p in model.parameters()):,}")
    
    
    # Сравнение эффективности
    print("\n" + "="*60)
    print("СРАВНЕНИЕ ЭФФЕКТИВНОСТИ")
    print("="*60)
    
    # Эффективность по параметрам
    def calculate_efficiency(model):
        total_params = sum(p.numel() for p in model.parameters())
        # Простая метрика: параметры на выходной канал
        return total_params
    
    eff = calculate_efficiency(model)
    
    print(f"CompactNet: {eff:,} параметров")
    
    return model



In [8]:
# Запускаем эксперимент
if __name__ == "__main__":
    model = run_parameter_experiment()

ЭКСПЕРИМЕНТ: КОНТРОЛЬ КОЛИЧЕСТВА ПАРАМЕТРОВ

1. Компактная сеть (CompactNet):
АНАЛИЗ КОЛИЧЕСТВА ПАРАМЕТРОВ
conv1          :     448 параметров
conv1x1        :     136 параметров
conv2          :   1,168 параметров
depthwise      :     160 параметров
pointwise      :     544 параметров
conv3          :  18,496 параметров
fc             :     650 параметров
--------------------------------------------------
Общее количество параметров: 21,602
Лимит параметров: 50,000
Использовано: 43.2% лимита
✓ Лимит параметров соблюден!


ТЕСТИРОВАНИЕ ПРОИЗВОДИТЕЛЬНОСТИ

CompactNet:
  Вход: torch.Size([4, 3, 32, 32])
  Выход: torch.Size([4, 10])
  Параметры: 21,874

СРАВНЕНИЕ ЭФФЕКТИВНОСТИ
CompactNet: 21,874 параметров
